In [14]:
import tensorflow as tf
import os
import random
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Conv2D,MaxPooling2D
from tensorflow.keras import Model

In [15]:
# pcap文件读取
class readData():
    def __init__(self, root="D:\code\pycode\DoHtunnel\dataset\pcaps_test", should_pad=False):
        self.root = root
        self.should_pad = should_pad
        self.pcaps = []
        num = 0
        for f in os.listdir(root):
            full_path = os.path.join(self.root, f)
            # drop the few outlier pcaps that were over 100KB
            if os.path.getsize(full_path) < 100_000:
                self.pcaps.append(full_path)
        self.max_len = 40000

    def __len__(self):
        return len(self.pcaps)

    def getitem(self, pcapath):
        # pcap = self.pcaps[idx]
        b=[]
        nums=0
        with open(pcapath, 'rb') as f:
            for i in f.read():
                b.append(int(i) / 255)
                nums+=1
                if(nums>=self.max_len):
                    break
        if self.should_pad and len(b)<self.max_len:
            # pad out to max len for consistency
            b += [0] * (self.max_len - len(b))
            
        b = np.array(b).reshape(200,200)
        b = np.expand_dims(b,axis=2)                
        return tf.constant(b)

In [16]:
#数据处理

datasetT=readData(root = 'D:\\code\\pycode\\DoHtunnel\\dataset\\finalData\\Benign2',should_pad=True)

dataX=[]
dataY=[]
num = 0
for pcap in datasetT.pcaps:
    x=datasetT.getitem(pcap)
    dataX.append(x)
    dataY.append(tf.constant(0))
print("SSSSSSSSSS")
datasetF=readData(root = 'D:\\code\\pycode\\DoHtunnel\\dataset\\finalData\\Malicious',should_pad=True)

for pcap in datasetF.pcaps:
    num+=1
    x=datasetF.getitem(pcap)
    dataX.append(x)
    dataY.append(tf.constant(1))
print(num)
dataC = list(zip(dataX, dataY))
random.shuffle(dataC)
dataX[:], dataY[:] = zip(*dataC)

lengthData=len(dataX)
train_rate=0.5
bj=int(lengthData*train_rate)
x_train = dataX[0:bj]
y_train = dataY[0:bj]
x_test = dataX[bj:lengthData]
y_test = dataY[bj:lengthData]


SSSSSSSSSS
771


In [55]:
lentrain=len(x_train)
lentest=len(x_test)

print(lengthData)
print(x_train[0])
for i in range(10):
    print(y_train[i])
#    print(y_train[i][0])

2413
tf.Tensor(
[[[0.83137255]
  [0.76470588]
  [0.69803922]
  ...
  [0.        ]
  [0.        ]
  [0.30980392]]

 [[0.49803922]
  [0.68627451]
  [0.25098039]
  ...
  [0.55686275]
  [0.59215686]
  [0.58039216]]

 [[0.74901961]
  [0.21568627]
  [0.45490196]
  ...
  [0.55294118]
  [0.67058824]
  [0.21960784]]

 ...

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]], shape=(200, 200, 1), dtype=float64)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=

In [18]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(2000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(2000).batch(32)

In [59]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(10,3,input_shape=(200,200,1),activation='relu')
    self.pooL1 = MaxPooling2D(pool_size=3,input_shape=(198,198,10))
    # maxlen//3 , 10
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(2)
    
  def call(self, x):
    x = self.conv1(x)
    x = self.pooL1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [60]:
#交叉熵
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [61]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
train_falsePositives = tf.keras.metrics.FalsePositives(name='train_falsePositives')
train_falseNegatives = tf.keras.metrics.FalseNegatives(name='train_falseNegatives')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
test_falsePositives = tf.keras.metrics.FalsePositives(name='test_falsePositives')
test_falseNegatives = tf.keras.metrics.FalseNegatives(name='test_falseNegatives')



In [62]:
@tf.function
def train_step(lines, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(lines, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)
  train_falsePositives(labels, tf.argmax(predictions, axis=1))
  train_falseNegatives(labels, tf.argmax(predictions, axis=1))

In [63]:
@tf.function
def test_step(lines, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(lines, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
  test_falsePositives(labels, tf.argmax(predictions, axis=1))
  test_falseNegatives(labels, tf.argmax(predictions, axis=1))

In [64]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for lines, labels in train_ds:
    train_step(lines, labels)

  for test_lines, test_labels in test_ds:
    test_step(test_lines, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'FalsePositives: {train_falsePositives.result()/lentrain}, '
    f'FalseNegatives: {train_falseNegatives.result()/lentrain}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}, '
    f'Test FalsePositives: {test_falsePositives.result()/lentest},'
    f'Test FalseNegatives: {test_falseNegatives.result()/lentest},'
  )



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.7151856422424316, Accuracy: 93.03482818603516, FalsePositives: 0.04477611929178238, FalseNegatives: 0.024875622242689133, Test Loss: 0.026944590732455254, Test Accuracy: 99.83429718017578, Test FalsePositives: 0.0,Test FalseNegatives: 0.0016570008592680097,
Epoch 2, Loss: 0.010875385254621506, Accuracy: 99.91708374023438, FalsePositives: 0.04477611929178238, FalseNegatives: 0.02570480853319168, Test Loss: 0.02191479317843914, Test Accuracy: 99.83429718017578, Test FalsePositives: 0.0,Te

In [65]:
print(model.summary())

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            multiple                  100       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_3 (Flatten)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  5575808   
_________________________________________________________________
dense_7 (Dense)              multiple                  258       
Total params: 5,576,166
Trainable params: 5,576,166
Non-trainable params: 0
_________________________________________________________________
None


In [66]:
num=0
for images , labels in train_ds:
    print(images.shape)
    tsf=model.call(images)
    print(tsf)
    for j in tsf:
        if(j[0]<j[1]) :
            print(1,end=' ')
        else:
            print(0,end=' ')
    print()
    print(labels)
    num+=1
    if(num==3):
        break

(32, 200, 200, 1)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[[-15.960386     3.4962916 ]
 [  7.6527996   -5.5337725 ]
 [-14.500354     2.4461055 ]
 [ 10.413767    -2.431469  ]
 [  9.423071    -3.3139477 ]
 [  9.692662    -3.0201561 ]
 [-13.917008     2.3067043 ]
 [-14.154809     2.4591775 ]
 [-14.698944     2.9266584 ]
 [  0.31188595  -7.7261033 ]
 [  4.1029515   -3.292359  ]
 [  5.883159   -10.195912  ]
 [ 11.153595   -10.374996  ]
 [  9.5020075   -2.4342904 ]
 [  4.2878838   -3.4246604 ]
 [  2.1597736   -8.963727  ]
 [-14.618707   